# RL for Autonomous Vehicle Tasks: Safety and Traffic Optimization

## Agents

In [244]:
# TODO: define agents here

## Scenarios

In [245]:
import gymnasium
import highway_env
from matplotlib import pyplot as plt
%matplotlib inline
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
import logging
import json
import os
import numpy as np
import tensorboard

### Car following

In [246]:
def speed_matching_reward(v_ego, v_lead):
    return 1.0 - abs(v_ego - v_lead) / max(v_lead, 1.0)  # Normalize to [0, 1].

def safe_distance_penalty(d_rel, v_rel):
    desired_distance = 2.0  # Optimal distance in meters.
    safe_margin = 1.0       # Allowable range around desired distance.
    if d_rel < desired_distance - safe_margin:  # Too close, penalize heavily
        return -1.0
    elif d_rel > desired_distance + safe_margin:  # Too far, minor penalty
        return -0.2
    else:  # Reward for being at a safe following distance
        return 0.5

def optimal_speed_range_reward(v_ego, v_lead, v_min=10, v_max=15):
    if (v_ego < v_min or v_ego > v_max) and v_lead > v_ego: 
        return -0.5
    else:
        return 0.2


def reward_function(env, ego_vehicle, lead_vehicle):
    d_rel = lead_vehicle.position[0] - ego_vehicle.position[0]
    v_rel = lead_vehicle.velocity[0] - ego_vehicle.velocity[0]

    if ego_vehicle.crashed:
        return -10.0
    
    if ego_vehicle.position[0] > env.road.network.get_lane(ego_vehicle.lane_index).length:
        return 10.0

    safe_distance_reward = safe_distance_penalty(d_rel, v_rel)
    speed_match_reward = speed_matching_reward(ego_vehicle.velocity[0], lead_vehicle.velocity[0])
    optimal_speed_range_reward = optimal_speed_range_reward(ego_vehicle.velocity[0], lead_vehicle.velocity[0])

    return safe_distance_reward + speed_match_reward + optimal_speed_range_reward

car_following = gymnasium.make('highway-v0', render_mode="rgb_array")
car_following.unwrapped.configure({
    "vehicles_count": 1,
    "controlled_vehicles": 1,
    "lanes_count": 1,
    "action": {
    "type": "DiscreteMetaAction"  # Uses a discrete action space supported by your algorithm.
    },
    "observation": {
        "type": "Kinematics",
        "vehicles_count": 2,
        "features": ["x", "y", "vx", "vy"],
        "absolute": False,
        "normalize": True
    },
    #"collision_reward": -10.0,
    #"reward_speed_range": [10, 15], 

    "screen_width": 600,             # Environment's graphical representation width.
    "screen_height": 600,
    "duration": 100,                  
    "offscreen_rendering": False, 
    "show_trajectories": True
})
car_following.reset()
car_following.reward_function = lambda obs, action: reward_function(car_following, car_following.vehicle, car_following.road.vehicles[1])

### Overtaking (2nd version)

In [247]:
def overtaking_reward_function(env, ego_vehicle, road_vehicles):
    d_rel = road_vehicles[0].position[0] - ego_vehicle.position[0]
    v_rel = road_vehicles[0].velocity[0] - ego_vehicle.velocity[0]
    ego_lane = ego_vehicle.lane_index

    # Reward for maintaining safe distance
    safe_distance_reward = safe_distance_penalty(d_rel, v_rel)
    
    # Reward for matching speed
    speed_match_reward = speed_matching_reward(ego_vehicle.velocity[0], road_vehicles[0].velocity[0])

    # Reward for staying within the optimal speed range (when not overtaking)
    optimal_speed_range_reward = optimal_speed_range_reward(ego_vehicle.velocity[0], road_vehicles[0].velocity[0])

    # Overtaking specific reward: Reward when agent overtakes and does it safely
    overtaking_reward = 0
    if ego_vehicle.position[0] > road_vehicles[0].position[0] and ego_vehicle.lane_index != ego_lane:
        overtaking_reward = 5  # Reward for overtaking successfully

    # **Reward for staying in the correct lane after overtaking:**
    correct_lane_reward = 0
    if ego_vehicle.lane_index == 1:  # Assuming lane index 1 is the right lane
        correct_lane_reward = 2  # Reward if staying in the right lane after overtaking
    
    # **Penalty for going into the wrong lane (unnecessary lane changes after overtaking):**
    incorrect_lane_penalty = 0
    if ego_vehicle.lane_index != 1 and ego_vehicle.position[0] > road_vehicles[0].position[0]:  # When in wrong lane after overtaking
        incorrect_lane_penalty = -2  # Small penalty for drifting into the wrong lane

    # Penalize for collisions during overtaking
    if ego_vehicle.crashed:
        return -10.0

    # Reward the agent for completing a successful overtake without crashing
    if ego_vehicle.position[0] > env.road.network.get_lane(ego_vehicle.lane_index).length:
        return 10.0

    # Combine all rewards
    total_reward = (safe_distance_reward + 
                    speed_match_reward + 
                    optimal_speed_range_reward + 
                    overtaking_reward + 
                    correct_lane_reward + 
                    incorrect_lane_penalty)

    return total_reward

car_overtaking = gymnasium.make('highway-v0', render_mode="rgb_array")
car_overtaking.unwrapped.configure({
    "vehicles_count": 5,
    "controlled_vehicles": 1,
    "lanes_count": 2,
    "action": {
        "type": "DiscreteMetaAction"
    },
    "observation": {
        "type": "Kinematics",
        "vehicles_count": 6,
        "features": ["x", "y", "vx", "vy"],  # Remove "lane"
        "absolute": False,
        "normalize": True
    },
    "screen_width": 600,
    "screen_height": 600,
    "duration": 300,
    "offscreen_rendering": False,
    "show_trajectories": True
})
car_overtaking.reset()

# Defining the reward function here (see below).
car_overtaking.reward_function = lambda obs, action: overtaking_reward_function(car_overtaking, car_overtaking.vehicle, car_overtaking.road.vehicles)


### Highway cruising

In [248]:
highway_cruising = gymnasium.make('highway-v0', render_mode="rgb_array")
highway_cruising.unwrapped.configure({
    "vehicles_count": 15,  # Increased vehicle count to make the traffic denser.
    "controlled_vehicles": 1,  # One controlled vehicle (ego vehicle).
    "lanes_count": 3,  # More lanes to test the agent's ability to overtake in traffic.
    "road_type": "straight",  # Use straight road to make traffic management simpler (but still complex).
    "vehicles_density": 1.0,  # Higher density for more vehicles.
    "action": {
        "type": "DiscreteMetaAction"  # The action space is discrete for your agent's maneuverability.
    },
    "observation": {
        "type": "Kinematics",
        "vehicles_count": 20,  # Including traffic vehicles.
        "features": ["x", "y", "vx", "vy"],  # Key kinematic features.
        "absolute": False,
        "normalize": True  # Normalize observations.
    },
    "screen_width": 800,  # Larger screen for a wider view of the environment.
    "screen_height": 800,  # Larger height for better vehicle representation.
    "duration": 500,  # Increase the duration to simulate longer driving periods.
    "offscreen_rendering": False,
    "show_trajectories": True,  # Keep track of trajectories to see pathing.
    "traffic_density": 1.5,  # Increased traffic density for better generalization.
    "highway_length": 2000  # Make the highway longer to simulate more complex navigation.
})

# Reset environment
highway_cruising.reset()

# Assign the reward function
highway_cruising.reward_function = lambda obs, action: overtaking_reward_function(highway_cruising, highway_cruising.vehicle, highway_cruising.road.vehicles)


### Lane Changing

In [249]:
def speed_matching_reward(v_ego, v_lead):
    return 1.0 - abs(v_ego - v_lead) / max(v_lead, 1.0)  # Normalize to [0, 1].

def safe_distance_penalty(d_rel, v_rel):
    desired_distance = 2.0  # Optimal distance in meters.
    safe_margin = 1.0       # Allowable range around desired distance.
    if d_rel < desired_distance - safe_margin:  # Too close, penalize heavily
        return -1.0
    elif d_rel > desired_distance + safe_margin:  # Too far, minor penalty
        return -0.2
    else:  # Reward for being at a safe following distance
        return 0.5


def reward_function(env, ego_vehicle, lead_vehicle):
    d_rel = lead_vehicle.position[0] - ego_vehicle.position[0]
    v_rel = lead_vehicle.velocity[0] - ego_vehicle.velocity[0]

    if ego_vehicle.crashed:
        return -10.0

    # Compute rewards
    safe_distance_reward = safe_distance_penalty(d_rel, v_rel)
    speed_match_reward = speed_matching_reward(ego_vehicle.velocity[0], lead_vehicle.velocity[0])

    return safe_distance_reward + speed_match_reward

env = gymnasium.make('highway-v0', render_mode="rgb_array")
env.unwrapped.configure({
    "vehicles_count": 2,
    "controlled_vehicles": 1,
    "lanes_count": 1,
    "action": {
    "type": "DiscreteMetaAction"  # Uses a discrete action space supported by your algorithm.
    },
    "observation": {
        "type": "Kinematics",
        "vehicles_count": 2,
        "features": ["x", "y", "vx", "vy"],
        "absolute": False,
        "normalize": True
    },
    "collision_reward": -10.0,
    "reward_speed_range": [5, 15], 

    "screen_width": 600,             # Environment's graphical representation width.
    "screen_height": 600,
    "duration": 20,                  
    "offscreen_rendering": False
})
env.reset()

env.reward_function = lambda obs, action: reward_function(env, env.vehicle, env.road.vehicles[1])

### Roundabout

In [250]:
roundabout_env = gymnasium.make('roundabout-v0', render_mode='rgb_array')

roundabout_config = {
    "vehicles_count": 30,            # Number of vehicles in the environment.
    "controlled_vehicles": 1,        # Number of vehicles controlled by the agent.
    "screen_width": 600,             # Environment's graphical representation width.
    "screen_height": 600,            # Environment's graphical representation height.
    "centering_position": [0.5, 0.5],# Centering the agent in the screen ([x, y]).
    "initial_spacing": 2,            # Initial spacing between vehicles.
    "duration": 11,                  # Time duration of an episode, in seconds.

    # Map Parameters
    "radius": 30.0,                  # Radius of the roundabout (inner circle radius).
    "lanes_count": 2,                # Number of lanes in the roundabout.
    "incoming_lanes": 1,             # Number of incoming lanes per road entering the roundabout.
    "outgoing_lanes": 1,             # Number of outgoing lanes per road exiting the roundabout.

    # Action and Reward Parameters
    "normalize_reward": True,        # Whether to normalize rewards to [0, 1].
    "collision_reward": -1.0,        # Reward penalty for collisions.
    "right_lane_reward": 0.1,        # Reward for driving in the correct lane.
    "high_speed_reward": 0.4,        # Reward for maintaining a high speed.
    "reward_speed_range": [20, 30],  # Speed range in which high-speed reward is maximized (in m/s).
    "forward_motion_reward": 0.1,    # Reward for moving forward.
    "exiting_reward": 1.0,           # Reward for exiting the roundabout.
    "low_speed_penalty": -0.2,       # Penalty for maintaining a low speed.


    # Observation Parameters
    "observation": {
        "type": "Kinematics",        # Type of observation (Kinematics provides position, velocity, etc.).
        "vehicles_count": 5,         # Number of surrounding vehicles considered in observation.
        "features": ["presence", "x", "y", "vx", "vy"], # Features to observe.
        "features_range": {
            "x": [-100, 100],        # x-coordinate range.
            "y": [-100, 100],        # y-coordinate range.
            "vx": [-20, 20],         # x-velocity range.
            "vy": [-20, 20],         # y-velocity range.
        },
        "absolute": False,           # Relative (False) or absolute (True) position values.
        "normalize": True,           # Whether to normalize the observation values.
    },

    # Rendering Parameters
    "offscreen_rendering": False,    # Whether rendering happens off-screen.
}

roundabout_env.unwrapped.configure(roundabout_config)
roundabout_env.reset()

(array([[ 1.        ,  0.02      ,  0.45      ,  0.        , -0.4       ],
        [ 1.        , -0.03989115, -0.2509916 ,  0.8049084 ,  0.48045164],
        [ 1.        , -0.20781857, -0.3005872 ,  0.45850477,  0.976361  ],
        [ 1.        ,  1.        , -0.47      , -0.8323256 ,  0.4       ],
        [ 1.        , -0.19635443, -0.5443351 , -0.32996246,  1.        ]],
       dtype=float32),
 {'speed': 8,
  'crashed': False,
  'action': 2,
  'rewards': {'collision_reward': False,
   'high_speed_reward': 0.5,
   'lane_change_reward': True,
   'on_road_reward': True}})

### Overtaking

In [251]:
overtaking_env = gymnasium.make('highway-v0', render_mode='rgb_array')
config = {
    "vehicles_count": 50,  # Number of vehicles in the environment
    "controlled_vehicles": 1,  # Number of vehicles controlled by the agent
    "duration": 30,  # Duration of each episode
    "collision_reward": -1,  # Stronger penalty for collisions
    "lane_change_reward": 0.5,  # Reward for staying in lane
    "right_lane_reward": 0.1,  # Reward for being in the right lane
    "overtaking_reward": 1,  # Reward for overtaking
    "reward_speed_range": [20, 30],  # Reward only within this speed range
    "simulation_frequency": 5,  # Lower simulation speed to avoid erratic behavior
    "policy_frequency": 1,  # Fewer policy updates per second
    "ego_spacing": 1.5,  # Desired minimum spacing between the ego vehicle and the vehicle in front
    "screen_width": 600,
    "screen_height": 400,
    "offscreen_rendering": False,
    "offroad_terminal": True,
    "show_trajectories": True,
    "render_agent": True,
    "action": {
        "type": "DiscreteAction"  # Discrete control (steer left, right, accelerate)
    },
    "observation": {
        "type": "Kinematics"
    },
}
overtaking_env.unwrapped.configure(config)
overtaking_env.reset()

(array([[ 1.        ,  0.6759145 ,  0.75      ,  0.3125    ,  0.        ],
        [ 1.        ,  0.10509241, -0.25      , -0.02595853,  0.        ],
        [ 1.        ,  0.22146922, -0.25      , -0.02518469,  0.        ],
        [ 1.        ,  0.32768154, -0.75      , -0.03088426,  0.        ],
        [ 1.        ,  0.43229875,  0.        , -0.03412271,  0.        ]],
       dtype=float32),
 {'speed': 25,
  'crashed': False,
  'action': 3,
  'rewards': {'collision_reward': 0.0,
   'right_lane_reward': 1.0,
   'high_speed_reward': 0.5,
   'on_road_reward': 1.0}})

## Training and Testing

In [252]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def create_directory(path):
    """
    Create directory if it doesn't exist
    
    Args:
        path (str): Directory path to create
    """
    try:
        os.makedirs(path, exist_ok=True)
        logger.info(f"Directory created: {path}")
    except Exception as e:
        logger.error(f"Error creating directory {path}: {e}")

def save_pipeline_data(agent_name, stage, environment_name, data):
    """
    Save pipeline stage data
    
    Args:
        agent_name (str): Name of the RL agent
        stage (str): Current pipeline stage
        environment_name (str): Name of the environment
        data (dict): Data to be saved
    """
    create_directory(f"results/{agent_name}")
    
    try:
        filename = f"results/{agent_name}/{environment_name}_{stage}_data.json"
        with open(filename, 'w') as f:
            json.dump(data, f, indent=4)
        logger.info(f"Data saved for {stage} in {environment_name}")
    except Exception as e:
        logger.error(f"Error saving data: {e}")

In [253]:
def train(agent, environment, agent_name, stage, environment_name, timesteps=1000):
    """
    Train the agent in a specific environment and save the model
    
    Args:
        agent (sb3.BaseAlgorithm): RL agent to train
        environment (gym.Env): Environment to train in
        agent_name (str): Name of the agent
        stage (str): Current pipeline stage
        environment_name (str): Name of the environment
        timesteps (int, optional): Number of training timesteps. Defaults to 100.
    
    Returns:
        Trained agent
    """
    try:
        # Wrap environment with Monitor
        log_dir = f"logs/{agent_name}"
        os.makedirs(log_dir, exist_ok=True)
        env = Monitor(environment, log_dir)
            
        # Set environment and learn
        agent.set_env(env)
        agent.learn(total_timesteps=timesteps)

        # Get training results
        training_results = {
            'rewards': env.get_episode_rewards(),
        }
        
        # Create directories for model and results
        create_directory(f"models/{agent_name}")
        
        # Save model
        agent.save(f"models/{agent_name}/{environment_name}_model")
        
        # Prepare and save training information
        training_info = {
            'agent_name': agent_name,
            'environment_name': environment_name,
            'stage': stage,
            'total_timesteps': timesteps,
            'training_results': training_results
        }
        save_pipeline_data(agent_name, "train", environment_name, training_info)
        
        logger.info(f"Training completed for {agent_name} in {environment_name}")
        return agent
    
    except Exception as e:
        logger.error(f"Training failed for {agent_name} in {environment_name}: {e}")
        raise



In [254]:
def test(agent, environment, agent_name, stage, environment_name, num_episodes=10):
    test_results = []
    total_rewards = []
    total_collisions = 0
    traffic_speeds = []

    for episode in range(num_episodes):
        try:
            obs, info = environment.reset()
            done = truncated = False
            episode_reward = 0
            episode_collisions = 0
            episode_speeds = []

            while not (done or truncated):
                # Predict action using the agent
                action, _states = agent.predict(obs, deterministic=True)
                obs, reward, done, truncated, info = environment.step(action)

                # Update reward
                episode_reward += reward
                
                # Check for collisions
                if info.get("crashed", False):
                    episode_collisions += 1

                # Gather traffic speeds
                current_speeds = [
                    vehicle.speed
                    for vehicle in environment.unwrapped.road.vehicles  # Use unwrapped
                    if vehicle != environment.unwrapped.vehicle  # Exclude agent vehicle
                ]
                episode_speeds.extend(current_speeds)

                # Optional rendering
                environment.render()

            environment.close()

            # Collect data for this episode
            total_rewards.append(episode_reward)
            total_collisions += episode_collisions
            traffic_speeds.extend(episode_speeds)

            test_results.append({
                'episode': episode,
                'total_reward': episode_reward,
                'collisions': episode_collisions,
                'avg_speed': np.mean(episode_speeds) if episode_speeds else 0,
                'speed_variance': np.var(episode_speeds) if episode_speeds else 0
            })
        
        except Exception as e:
            logger.error(f"Test episode {episode + 1} failed in {environment_name}: {e}")

    # Calculate overall KPIs
    avg_reward = np.mean(total_rewards)
    avg_speed = np.mean(traffic_speeds) if traffic_speeds else 0
    speed_variance = np.var(traffic_speeds) if traffic_speeds else 0

    logger.info(f"Test completed in {environment_name} - "
                f"Avg Reward: {avg_reward}, Total Collisions: {total_collisions}, "
                f"Avg Traffic Speed: {avg_speed}, Speed Variance: {speed_variance}")

    # Save test results and KPIs
    save_pipeline_data(agent_name, stage, environment_name, {
        'test_results': test_results,
        'kpis': {
            'average_reward': avg_reward,
            'total_collisions': total_collisions,
            'average_speed': avg_speed,
            'speed_variance': speed_variance
        },
        'environment': environment_name,
        'stage': stage
    })

    return {
        'test_results': test_results,
        'kpis': {
            'average_reward': avg_reward,
            'total_collisions': total_collisions,
            'average_speed': avg_speed,
            'speed_variance': speed_variance
        }
    }

In [255]:
def rl_pipeline(initial_agent, agent_name, environments):
    """
    Sequential environment training pipeline with customizable order
    
    Args:
        initial_agent (sb3.BaseAlgorithm): Initial RL agent
        agent_name (str): Name of the agent
        environments (list of tuples): A list of (env_name, gym.Env) tuples defining the order of execution.
            Example: [("lane_changing", lane_changing_env), ("roundabout", roundabout_env), ("overtaking", overtaking_env)]
    
    Returns:
        Trained agent
    """

    logger.info("Customizable Sequential Environment Pipeline Started")
    
    current_agent = initial_agent

    for i, (env_name, env) in enumerate(environments):
        logger.info(f"Stage {i + 1}: {env_name.capitalize()} Environment")
        
        if i > 0:  # Test the previous agent in the current environment before training
            previous_env_name = environments[i - 1][0]
            logger.info(f"Testing {previous_env_name.capitalize()} Agent in {env_name.capitalize()}")
            test(
                current_agent, env, 
                agent_name, 'pre_training_test', env_name
            )
        
        # Train in the current environment
        logger.info(f"Training in {env_name.capitalize()} Environment")
        current_agent = train(
            current_agent, env, 
            agent_name, env_name, env_name, 50000
        )
        
        # Test in the current environment
        logger.info(f"Testing in {env_name.capitalize()} Environment")
        test(
            current_agent, env, 
            agent_name, 'pos_training_test', env_name
        )

    logger.info("Customizable Sequential Environment Pipeline Completed")

    # Save the final agent
    create_directory(f"models/{agent_name}/final")
    current_agent.save(f"models/{agent_name}/{agent_name}_final_model")
    
    return current_agent


In [256]:
# %tensorboard --logdir "agent_DQN"

- ### Agent 1

In [257]:
agent_DQN = DQN('MlpPolicy', highway_cruising,
        policy_kwargs=dict(net_arch=[256, 256]),
        learning_rate = 1e-4,
        buffer_size=15000,
        learning_starts=200,
        batch_size=32,
        gamma=0.8,
        train_freq=1,
        gradient_steps=1,
        target_update_interval=50,
        verbose=1,
        tensorboard_log="agent_DQN/")

final_agent_DQN = rl_pipeline(agent_DQN, 'agent_DQN', [("highway_cruising", highway_cruising)])#("lane_changing", lane_changing_env), ("roundabout", roundabout_env), ("overtaking", overtaking_env)])


""" Maybe compare to this! """

# # Training order is lane changing -> roundabout -> overtaking
# final_agent_DQN_lro = rl_pipeline(agent_DQN, 'agent_DQN_lro', [("lane_changing", lane_changing_env), ("roundabout", roundabout_env), ("overtaking", overtaking_env)])

# # Training order is roundabout -> overtaking -> lane changing
# final_agent_DQN_rol = rl_pipeline(agent_DQN, 'agent_DQN_rol', [("roundabout", roundabout_env), ("overtaking", overtaking_env), ("lane_changing", lane_changing_env)])

# # Training order is overtaking -> lane changing -> roundabout
# final_agent_DQN_olr = rl_pipeline(agent_DQN, 'agent_DQN_olr', [("overtaking", overtaking_env), ("lane_changing", lane_changing_env), ("roundabout", roundabout_env)])

# # Training order is overtaking -> roundabout -> lane changing
# final_agent_DQN_orl = rl_pipeline(agent_DQN, 'agent_DQN_orl', [("overtaking", overtaking_env), ("roundabout", roundabout_env), ("lane_changing", lane_changing_env)])

# # Training order is roundabout -> lane changing -> overtaking
# final_agent_DQN_rlo = rl_pipeline(agent_DQN, 'agent_DQN_rlo', [("roundabout", roundabout_env), ("lane_changing", lane_changing_env), ("overtaking", overtaking_env)])

# # Training order is lane changing -> overtaking -> roundabout
# final_agent_DQN_lor = rl_pipeline(agent_DQN, 'agent_DQN_lor', [("lane_changing", lane_changing_env), ("overtaking", overtaking_env), ("roundabout", roundabout_env)])

2024-12-18 20:07:07,163 - INFO - Customizable Sequential Environment Pipeline Started
2024-12-18 20:07:07,163 - INFO - Stage 1: Highway_cruising Environment
2024-12-18 20:07:07,164 - INFO - Training in Highway_cruising Environment


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a DummyVecEnv.
Logging to agent_DQN/DQN_40
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 16.5     |
|    ep_rew_mean      | 12.5     |
|    exploration_rate | 0.987    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 33       |
|    time_elapsed     | 1        |
|    total_timesteps  | 66       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 15.2     |
|    ep_rew_mean      | 11.7     |
|    exploration_rate | 0.977    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 33       |
|    time_elapsed     | 3        |
|    total_timesteps  | 122      |
----------------------------------
----------------------------------
| rollout/            |          |
|  

2024-12-18 20:36:32,224 - INFO - Directory created: models/agent_DQN
2024-12-18 20:36:32,229 - INFO - Directory created: results/agent_DQN
2024-12-18 20:36:32,231 - INFO - Data saved for train in highway_cruising
2024-12-18 20:36:32,231 - INFO - Training completed for agent_DQN in highway_cruising
2024-12-18 20:36:32,231 - INFO - Testing in Highway_cruising Environment
2024-12-18 20:38:00,479 - INFO - Test completed in highway_cruising - Avg Reward: 52.34046174571411, Total Collisions: 10, Avg Traffic Speed: 21.03916974209624, Speed Variance: 1.6290399176646244
2024-12-18 20:38:00,479 - INFO - Directory created: results/agent_DQN
2024-12-18 20:38:00,480 - INFO - Data saved for pos_training_test in highway_cruising
2024-12-18 20:38:00,480 - INFO - Customizable Sequential Environment Pipeline Completed
2024-12-18 20:38:00,480 - INFO - Directory created: models/agent_DQN/final


' Maybe compare to this! '

- ### Agent N

## Performance

In [258]:
# Display results
def display_results(agent_name, stages, type="table"):
    # Load results
    results = {}
    for stage in stages:
        with open(f"results/{agent_name}/{stage}_data.json", 'r') as f:
            results[stage] = json.load(f)

    if type == "list":
        print(f"--- {agent_name} ---\n")
        for stage, data in results.items():
            print(f"> {stage}")
            print(f"    - Average Reward: {data['kpis']['average_reward']}")
            print(f"    - Total Collisions: {data['kpis']['total_collisions']}")
            print(f"    - Average Speed: {data['kpis']['average_speed']}")
            print(f"    - Speed Variance: {data['kpis']['speed_variance']}")
            print("")
        print("--------------------\n")
    elif type == "table":
        table = []
        for stage, data in results.items():
            table.append([stage, data['kpis']['average_reward'], data['kpis']['total_collisions'], data['kpis']['average_speed'], data['kpis']['speed_variance']])
        table = np.array(table)

        print("|".join([f"{{[{str(agent_name)}]:<20}} {str('Stage'):>10} ", "Average Reward".center(20), "Total Collisions".center(20), "Average Speed".center(20), "Speed Variance".center(20)]))
        print("=" * 117)
        for row in table:
            formatted_row = [
                f"{str(row[0]):>31} ",    # Stage (left aligned)
                f"{str(row[1]):^20}",    # Average Reward (center aligned)
                f"{str(row[2]):^20}",    # Total Collisions (center aligned)
                f"{str(row[3]):^20}",    # Average Speed (center aligned)
                f"{str(row[4]):^20}",     # Speed Variance (center aligned)
            ]
            print("|".join(formatted_row))
            print("-" * 117)
        print()
        

In [260]:
display_results("agent_DQN", ["lane_changing_pos_training_test", "roundabout_pre_training_test", "roundabout_pos_training_test", "overtaking_pre_training_test", "overtaking_pos_training_test"])

{[agent_DQN]:<20}      Stage |   Average Reward   |  Total Collisions  |   Average Speed    |   Speed Variance   
lane_changing_pos_training_test | 19.809523809523807 |         0          | 21.490179070718053 | 2.5431416061097925 
---------------------------------------------------------------------------------------------------------------------
   roundabout_pre_training_test |        5.0         |         8          | 13.176681440042938 | 17.91196936083314  
---------------------------------------------------------------------------------------------------------------------
   roundabout_pos_training_test | 8.192857142857145  |         4          |  15.1014806360992  | 9.968366355891002  
---------------------------------------------------------------------------------------------------------------------
   overtaking_pre_training_test | 9.662684444444448  |         8          | 20.412763661430365 | 2.3286179404918492 
----------------------------------------------------------------

In [261]:
# Test the final agent in all environments
def test_agent_all_envs(agent_name, environments, environment_names):
    # Load the final agent
    agent = DQN.load(f"models/{agent_name}/{agent_name}_final_model", device="cpu")

    # Lane Changing
    performance_lane_changing = test(agent, environments[0], agent_name, '', "final_test_" + environment_names[0])

    # Roundabout
    performance_roundabout = test(agent, environments[1], agent_name, '', "final_test_" + environment_names[1])

    # Overtaking
    performance_overtaking = test(agent, environments[2], agent_name, '', "final_test_" + environment_names[2])


In [262]:
test_agent_all_envs("agent_DQN", [highway_cruising], ["highway_cruising"])#[lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])


# test_agent_all_envs("agent_DQN_rol", [lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])
# test_agent_all_envs("agent_DQN_olr", [lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])
# test_agent_all_envs("agent_DQN_orl", [lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])
# test_agent_all_envs("agent_DQN_rlo", [lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])
# test_agent_all_envs("agent_DQN_lor", [lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])


2024-12-18 22:24:33,968 - INFO - Test completed in final_test_highway_cruising - Avg Reward: 38.575118151887736, Total Collisions: 10, Avg Traffic Speed: 21.05996506579231, Speed Variance: 1.6943530190097773
2024-12-18 22:24:33,969 - INFO - Directory created: results/agent_DQN
2024-12-18 22:24:33,970 - INFO - Data saved for  in final_test_highway_cruising


IndexError: list index out of range

## Analysis

In [20]:
# Display final test results
display_results("agent_DQN", ["final_test_lane_changing_", "final_test_roundabout_", "final_test_overtaking_"])

{[agent_DQN]:<20}      Stage |   Average Reward   |  Total Collisions  |   Average Speed    |   Speed Variance   
      final_test_lane_changing_ |      16.4125       |         4          | 20.602228832201057 | 1.7628795927127507 
---------------------------------------------------------------------------------------------------------------------
         final_test_roundabout_ |       12.05        |         0          | 14.381887780369855 | 19.82951714419477  
---------------------------------------------------------------------------------------------------------------------
         final_test_overtaking_ | 19.708888888888897 |         3          | 20.647436386620896 | 1.9794265486352098 
---------------------------------------------------------------------------------------------------------------------



## Conclusion